# Using the Eikon Data API

Any Eikon end user, where the data doesn’t leave the desktop, can use this API to access their Eikon data more flexibly.

## Eikon API Authentication: Using the APP KEY Application

Before you can use the Eikon API, you must first create an Application Key in Eikon .This key is used for Authentication, proof that you are who you say you are and that you are permissioned to view the data you are seeking to retrieve. The application is linked to your Eikon account, so whatever permissions you have on Eikon to view certain data will also apply to what you can view using the API (if that data is available via the API). 

#### Open the APP KEY Application in Eikon or Refinitiv Workspace.

To create a new Application Key, under Register Application, enter a name for your Application key in "App Display Name". 

Additionally, Check all 3 boxes: "Eikon Data API", "Side by Side API", "EDP API". 

Finally, click "Register New App" to complete creating your App Key for the Eikon DAPI.

The App Key will be generated below "Manage Applications"

### Best Eikon Apps for Content Help

1. Developer Community (DEVCOMM) 
2. Data Item Browser (DIB) 
3. Related Securities (RLTD)
4. What is this Ric? (META) 

The following is a useful guide on how to uncover the metadata behind Eikon Excel templates.
https://tinyurl.com/tvygoav

# Accessing the API in Python

First, you must import the eikon and pandas libraries into your IDE. I have imported others that prove to be useful for anaylsis after using the API

In [ ]:
import eikon as ek
import numpy as np
import pandas as pd
import configparser as cp

### Authenticating your App Key

Next, you must pass your App Key to be able to use the API. 

Use the set_app_key function, passing your App Key as a string as your only parameter. Remember, because this is an imported library, you must use "ek." to use Eikon libary functions

In [ ]:
app_key = "YOUR APP KEY HERE"

ek.set_app_key(app_key)

You can confirm that the function was successful if you do not receive an error messsage, but the get_app_key() function can help you verify as well

In [ ]:
ek.get_app_key()

## Understanding the many Eikon API Endpoints

The Eikon library has several functions that are various endpoints used to access the different datasets within Eikon.

The 5 most important functions we will explore today are:

1. get_symbology
2. get_data
3. get_timeseries
4. get_news_headlines
5. get_news_story

But there are several other functions that deal with the more technical settings of the API (set_port_number, get_json_request, etc.) that can be found in the API Documentation

### 1. get_symbology

get_symbology returns a list of instrument names converted into another instrument code

In [ ]:
help(ek.get_symbology)

In [ ]:
RICS = ["AAPL.O","GOOG.O","TSLA.O"]

symbology = ek.get_symbology(RICS)
symbology

symbology

In [ ]:
# Try it out.
# Create a list of RICS, of both Equities and other instruments.
# Use the get_symbology function to return different identifiers for the list.


In [ ]:
# Try it Out.
# Use additional parameters to_symbol_type and from_symbol_types



### Returns from the Eikon API and optional parameters

Eikon API uses Pandas Dataframes as the default return value. All reutrns from any request by default will be stored in a Pandas Dataframe. This is for simplicity of viewing your data. The alternative is to view the data in json formats.

While also easy to use, large data sets can get confusing when stored in a JSON format. However, if that is required, you can use the optional parameter raw_output. This is a boolean parameter that is by default set to False, which returns a pandas dataframe. Use "raw_output = True" in any of the five functions to get your data request in JSON format

In [ ]:
# Try it out. Use the get_symbology function, but set the raw_output parameter to True.
# What happened to the output?

In [ ]:
#Pandas Dataframe columns are Pandas series when you pull them out.
#If you want to convert your output into a list, use .tolist()

#Try it out. Convert the output of your get_symbology call to a list.


## 2. get_timeseries

This function allows you to retrieve historical data on instruments. 

For this function, you will need the following parameters

Required:
1. rics
2. fields - Available fields are found in using the help function.
3. start_date - in a string or datetime format (https://docs.python.org/3/library/datetime.html)

Optional
1. end_date if you do not put an end_date, the default is datetime.now()
2. interval - possible values found in help
3. count - max number of data points to retrieve
4. calendar
5. corax
6. normalize
7. raw_output
8. debug

In [ ]:
help(ek.get_timeseries)

In [ ]:
RICS = ["AAPL.O","GOOG.O","TSLA.O"]

timeseriesdata = ek.get_timeseries(RICS,
                                   start_date = "2019-03-15",
                                   end_date = "2020-01-17"
                                   )

In [ ]:
# Try it out. Use the get_timeseries function, specifying only the start_date and end_date parameters. 
# What is the output? What is the frequency of the data?


In [ ]:
#Try it out. Use get_timeseries for the same period, but now specify which fields to return, as well as a frequency.




## 3. get_data

This is arguably the most used function in this library. This function can be used to retrieve items found in the Data Item Browser. This function provides current and historical (when available) data.

You can retrieve a variety of information using this function: Company Name, ESG Scores, Latest Bid and Ask Price, Market Cap, CEO, Fundamentals, Z-Spread, G-Spread, Technical Analysis, etc.


In [ ]:
help(ek.get_data)

In [ ]:
data,err = ek.get_data(RICS,"TR.PriceClose")
data

In [ ]:
# Try it out. Use the get_data function to retrieve "TR.PriceClose" for your list of securities.




You can use lists for both instruments and fields

In [ ]:
#Try it out Use get_data to retrieve TRBC Business sector, EBITDA, and CF_ASK for your list of securities.



In [ ]:
data_1

In [ ]:
err1

### Adding Parameters to your request

#### Method 1: DIB Syntax

This is useful is you want to specify different parameters for each data item. Additionally, using this method allows you to copy and paste the exact syntax that is created in the bottom right corner of the DIB. 

In [ ]:
data_1, err1 = ek.get_data(RICS,
                           ["TR.TRBCBusinessSector",
                            "TR.EBITDA(Period=FY0,Curn=CAD,Scale=6)",   ## THIS LINE HERE
                            "CF_ASK"]
                          )

#### Method 2: Global Parameters

After listing fields, a dictionary of parameters for all fields can be added to modify your request. To request for past values of those data items that have timeseries data, SDate, EDate, and Frq can be used.

In [ ]:
ts,e = ek.get_data(RICs,["TR.PriceClose.date","TR.PriceClose",
                         "TR.TotalReturn1Mo"],
                       {'SDate':start_date,'EDate':end_date,'Frq':'D'})
ts

#### Method 3:  Dictionary of Parameters

Another alternative method (not preferred) is to modify the data item using dictionaries. Inside the fields parameter, to add parameters to a specific item, Create a dictionary where the key is the Data Item code, and the value is another dictionary whose first key is "params". The values of this dictionary will be another dictionary whose key values are the parameter names and the values are the values for the parameter you want. The additional keys in the second dictionary can be parameters regarding how to sort the columns according to this data item.

As you can see this process can become very complicated and confusing. It is recommended the first alternative method is used.

In [ ]:
data_3,err3 = ek.get_data(Stock_RICS,
                          ["TR.TRBCBusinessSector",
                           "TR.EBITDA",
                           "CF_ASK",{"TR.GrossProfit":{"params":{"Scale": 6, "Curn": "USD"},"sort_dir": "asc"}} ## HERE
                          ])

data_3

#### .set_index

set_index is in the pandas dataframe. set_index will Set the DataFrame index (row labels) using one or more existing columns or arrays (of the correct length). The index can replace the existing index or expand on it.

In [ ]:
data_3.set_index("Instrument")

## get_news_headlines and get_news_story

In [ ]:
help(ek.get_news_headlines)

In [ ]:
headlines = ek.get_news_headlines("TRUMP OR SANDERS OR WARREN AND ECONOMY AND Language:ENG", count=100,date_from="2019-12-01",date_to="2019-12-25")

headlines

In [ ]:
help(ek.get_news_story)

In [ ]:
news_story = ek.get_news_story("urn:newsml:reuters.com:20191224:nNRAairkb4:1",raw_output=True)

news_story

# Eikon Data API Usage and Limits Guidelines

https://developers.refinitiv.com/eikon-apis/eikon-data-api/docs?content=49692&type=documentation_item



## What Can and Can't Our Customers do with the API

The API is intended for an Eikon end user, meaning the data DOESN'T LEAVE THE DESKTOP!

Eikon users may not share their login credentials, run any instances of Eikon on a server or use, distribute or redistribute information in any way that is inconsistent with their organization’s agreement with us. 

Non-Display Use Generally Includes, But Is Not Limited To:

1. Use of data for the creation of new original works/derived data;
2. Use of data to create indices;
3. Use of data in electronic trading systems for the purpose of generating orders or executing transactions in an automated and/or semi-automated manner without the display of the original data. This includes, for instance, the use of the data for algorithmic trading, program trading or an automated monitoring of trading activities;
4. Use of data in risk management systems and portfolio management applications.

If you wish to resell or redistribute any applications which you develop using the Eikon Data API to third parties, you must enter into a developer's license agreement with us.